<a href="https://colab.research.google.com/github/vrra/Students-mentoring-2022/blob/main/Build-a-thon-2022/build_a_thon_graph_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created: 14 Apr 2022.   
Contact: Vishnu Ram OV.   
vishnu.n@ieee.org.   
Licence: available for all purposes under the sun


Refer: https://neo4j.com/docs/aura/auradb/getting-started/create-database/

step-1: install neo4j

In [1]:
pip install neo4j


     |████████████████████████████████| 90 kB 81 kB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for neo4j: filename=neo4j-4.4.3-py3-none-any.whl size=116070 sha256=fa1f755066cfb694f856d5d17faab014c731b8262f4373f12476c0c1ecb7711d
  Stored in directory: /Users/vishnu/Library/Caches/pip/wheels/90/fd/09/1de626b67971e1769f5e1f09c532f6a7117cc1b12c58d41601
Successfully built neo4j
Note: you may need to restart the kernel to use updated packages.


step-2: imports

In [2]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

step-3: define helper class

In [5]:


class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_friendship(self, actor1_name, actor2_name):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_friendship, actor1_name, actor2_name)
            for row in result:
                print("Created friendship between: {n1}, {n2}".format(n1=row['n1'], n2=row['n2']))

    @staticmethod
    def _create_and_return_friendship(tx, actor1_name, actor2_name):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "CREATE (n1:Actor { name: $actor1_name }) "
            "CREATE (n2:Actor { name: $actor2_name }) "
            "CREATE (n1)-[:USECAP]->(n2) "
            "RETURN n1, n2"
        )
        result = tx.run(query, actor1_name=actor1_name, actor2_name=actor2_name)
        try:
            return [{"n1": row["n1"]["name"], "n2": row["n2"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_actor(self, actor_name):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_actor, actor_name)
            for row in result:
                print("Found actor: {row}".format(row=row))

    @staticmethod
    def _find_and_return_actor(tx, actor_name):
        query = (
            "MATCH (n:Actor) "
            "WHERE n.name = $actor_name "
            "RETURN n.name AS name"
        )
        result = tx.run(query, actor_name=actor_name)
        return [row["name"] for row in result]

step-4: connect to DB

In [8]:
# Aura queries use an encrypted connection using the "neo4j+s" URI scheme
uri = "neo4j+s://a4270446.databases.neo4j.io"
user = "neo4j"
password = "Pn8T12EjCxRLRfaHClsJIaCFAdWcER4pEeIp4dRKyzg"
app = App(uri, user, password)

Step-5: Create relations

In [9]:
app.create_friendship("Experimentation Controller", "Sandbox")


Created friendship between: Experimentation Controller, Sandbox


Step-6: Query actors

In [10]:
app.find_actor("Experimentation Controller")

Found actor: Experimentation Controller


Step-6: Close the database connection

In [ ]:
app.close()